# Load libraries

In [1]:
import numpy as np
import pandas as pd
from pandasql import sqldf

# Configurations & Constants

In [2]:
user = 'participant1' # participant1-10
target_freq_as_int = 15 # 15|1
target_freq_unit1 = 'min' # min|s
target_freq_unit2 = 'm' # m|s
dataset_type = '' # '' | time_series_

In [3]:
wo_columns = {
    "Timestamp": "timestamp",
    "Heart Rate (in Beats per minute)": "heart_rate",
    "Stress Score": "stress_score",
    "Stress Interpretation": "stress_level",
    "Number of Steps": "steps",
    "Wearing Off": "wearing_off",
    "started_at": "wo_start",
    "finished_at": "wo_end",
    "Tremors": "wo_tremors",
    "Slowing down of movement": "wo_slowdown",
    "Change in mood or depression": "wo_moodchange",
    "Rigidity of muscles": "wo_rigidity",
    "Sharp pain or prolonged dull pain": "wo_pain",
    "Impairment of complex movements of the hand and fingers": "wo_impairment_hands",
    "Difficulty integrating thoughts or slowing down of thought": "wo_slow_thoughts",
    "Anxiety or panic attacks": "wo_anxiety",
    "Muscle spasm": "wo_muscle_spasm",
    "activity_target.activity_id": "report_id"
}

# Combine Datasets
Match wearing-off to combined Garmin data based on wearing-off start and end

In [4]:
garmin_data = pd.read_excel(f'./garmin_preprocessed.xlsx', sheet_name='garmin', engine='openpyxl')
wearing_off_with_symptoms = pd.read_excel(f'./fonlog_preprocessed.xlsx', sheet_name='fonlog', engine='openpyxl')

In [5]:
pysqldf = lambda q: sqldf(q, globals())
cond_join= '''
    select 
        garmin.*,
        wearing_off_with_symptoms.*,
        case
            when wearing_off_with_symptoms.[started_at] is not null THEN 1
        else 0
        end as 'Wearing Off'
    from garmin_data as garmin
    left join wearing_off_with_symptoms
    on garmin.[Timestamp] BETWEEN wearing_off_with_symptoms.[started_at] AND wearing_off_with_symptoms.[finished_at]
'''

# Change wearing-off columns
combined_data = pysqldf(cond_join).rename(columns=wo_columns)

# Drop duplicates based on timestamp
combined_data = combined_data.drop_duplicates(subset=['timestamp'])

# Set timestamp as index
combined_data['timestamp'] = pd.to_datetime(combined_data['timestamp'])
combined_data = combined_data.set_index('timestamp')

Compute for wearing-off duration

In [6]:
combined_data['wo_duration'] = ''
combined_data['wo_duration'] = (
    pd.to_datetime(combined_data.index) - pd.to_datetime(combined_data['wo_start'])
) / np.timedelta64(1, target_freq_unit2)

# gid = combined_data['wo_duration'].notnull().cumsum()
# dg = combined_data.groupby(gid)
# base = dg['wo_duration'].transform('last')
# combined_data['wo_duration'] = (  base + ( dg.cumcount() ) * target_freq_as_int)

# display(combined_data.iloc[140:150, ])

combined_data.to_excel(f'./combined.xlsx', sheet_name='combined')